Force de gravitation (Terre) :
$$
\ddot r = \frac{GM}{\mathbf{r}^3} \mathbf{r}
\quad\Rightarrow\quad
\begin{cases}
\ddot x = x\dfrac{GM}{\left(x^2+y^2+z^2\right)^\frac32}\\
\ddot y = y\dfrac{GM}{\left(x^2+y^2+z^2\right)^\frac32}\\
\ddot z = z\dfrac{GM}{\left(x^2+y^2+z^2\right)^\frac32}
\end{cases}
$$

Influence du soleil :
$$
\ddot r = \frac{GM\mathbf r}{\mathbf s^3}\big(-\mathbf{e}_r+3\mathbf{e}_s(\mathbf{e}_s\mathbf{e}_r)\big)
\quad\Rightarrow\quad
\begin{cases}
\ddot x = \\
\ddot y = \\
\ddot z = 
\end{cases}
$$

Influence de la lune :
$$
\ddot r = \frac{GM\mathbf r}{\mathbf l^3}\big(-\mathbf{e}_r+3\mathbf{e}_l(\mathbf{e}_l\mathbf{e}_r)\big)
\quad\Rightarrow\quad
\begin{cases}
\ddot x = \\
\ddot y = \\
\ddot z = 
\end{cases}
$$

Trainée atmosphérique :
$$
 \ddot{r} = -\frac{1}{2} C_D \frac{A}{m} \rho_0 e^{-h/H_0} \left(\dot r - \omega r\right)^2 \textbf{e}_v
 \quad\Rightarrow\quad
\begin{cases}
\ddot x = \\
\ddot y = \\
\ddot z = 
\end{cases}
$$

Géopotentiel :
$$
\begin{align*}
\ddot{r} &= \nabla\frac{GM}{r} + \nabla\frac{GM}{r} \frac{R^2}{r^2} \sqrt{5} \frac{1}{2}\left(3 \sin^2\varphi - 1\right) \bar{C}_{2 ,0} + \nabla\frac{GM}{r}\frac{R^2}{r^2} 3\cos^2\varphi \sqrt \frac{5}{12} \left(\bar{C}_{2,2} \cos(2\lambda) + \bar{S}_{2,2} \sin(2\lambda)\right)\\
&\Rightarrow\quad
\begin{cases}
\ddot x = \\
\ddot y = \\
\ddot z = 
\end{cases}
\end{align*}
$$